Function definitions for cluster-first route-second (CFRS) and route-first
cluster-second (RFCS) algorithms.

In [ ]:
from math import ceil
import numpy as np
from scipy.cluster.vq import kmeans2, whiten
#from TSPlib import nearest_neighbor
from ipynb.fs.full.TSPlib import nearest_neighbor, savings

In [ ]:
from scipy.cluster.vq import kmeans2, whiten

def CFRS(k, coordinates, d, plt, demands, vehicle_capacity):
    '''
    CFRS: Cluster First (KMeans), Route Second (NN), with capacity feasibility
    '''
    from ipynb.fs.full.TSPlib import nearest_neighbor

    depot = 0
    customers = list(range(1, len(coordinates)))
    coords = np.array(coordinates)

    # Step 1: Cluster using KMeans (based on location only)
    x, labels = kmeans2(whiten(coords[1:]), k, iter=100)
    labels = list(labels)

    # Step 2: Assign customers to clusters
    raw_clusters = [[] for _ in range(k)]
    for i, label in enumerate(labels):
        raw_clusters[label].append(i + 1)  # +1 to offset depot

    # Step 3: Sub-cluster by demand capacity
    clusters = []
    for cluster in raw_clusters:
        current_cluster = []
        current_demand = 0
        for customer in cluster:
            if current_demand + demands[customer] <= vehicle_capacity:
                current_cluster.append(customer)
                current_demand += demands[customer]
            else:
                clusters.append(current_cluster)
                current_cluster = [customer]
                current_demand = demands[customer]
        if current_cluster:
            clusters.append(current_cluster)

    # Step 4: Route each cluster with Nearest Neighbor
    vrp_solution = []
    total_length = 0
    for cluster in clusters:
        tour, _ = nearest_neighbor(cluster, depot, d)

        # Ensure the tour starts and ends at the depot only once
        if tour[0] != 0:
            tour = [0] + tour
        if tour[-1] != 0:
            tour = tour + [0]

        vrp_solution.append(tour)
        total_length += sum(d[tour[i]][tour[i+1]] for i in range(len(tour)-1))

    # Step 5: Plot
    colors = plt.cm.get_cmap("tab20", len(vrp_solution))
    for i, route in enumerate(vrp_solution):
        x = [coords[n][0] for n in route]
        y = [coords[n][1] for n in route]
        plt.plot(x, y, marker='o', color=colors(i))
    plt.axis('off')
    plt.title("CFRS Solution (Feasible)")
    plt.show()

    return vrp_solution, round(total_length, 2)


In [ ]:
from math import ceil
from ipynb.fs.full.TSPlib import christofides, nearest_neighbor

def RFCS(coordinates, d, demands, vehicle_capacity, method='christofides'):
    '''
    RFCS: Route-First, Cluster-Second using either Christofides or Nearest Neighbor,
    followed by capacity-based splitting.

    method: 'christofides' or 'nearest_neighbor'
    '''
    depot = 0
    customer_nodes = list(range(1, len(demands)))

    # Step 1: Generate tour over customer nodes
    if method == 'christofides':
        sub_d = d[np.ix_(customer_nodes, customer_nodes)]
        tsp_tour = christofides(sub_d)
        giant_tour = [customer_nodes[i] for i in tsp_tour[:-1]]  # map back to global indices
    elif method == 'nearest_neighbor':
        full_nodes = list(range(1, len(demands)))  # exclude depot
        tsp_tour, _ = nearest_neighbor(full_nodes, depot, d)
    # Ensure all customers are included
        giant_tour = [node for node in tsp_tour if node != depot]


    else:
        raise ValueError("Invalid method. Use 'christofides' or 'nearest_neighbor'.")

    # Step 2: Split into capacity-feasible routes
    vrp_solution = []
    current_route = [depot]
    current_demand = 0

    for customer in giant_tour:
        demand = demands[customer]
        if current_demand + demand <= vehicle_capacity:
            current_route.append(customer)
            current_demand += demand
        else:
            current_route.append(depot)
            vrp_solution.append(current_route)
            current_route = [depot, customer]
            current_demand = demand

    if current_route[-1] != depot:
        current_route.append(depot)
        vrp_solution.append(current_route)

    # Step 3: Calculate total distance
    total_length = sum(d[route[i]][route[i+1]] for route in vrp_solution for i in range(len(route)-1))
    total_length = round(total_length, 2)

    return vrp_solution, total_length
